In [183]:
from sage.all import *
import numpy as np
import random

def kernPause(a1,a2):
    return  1*(a1==a2)

def kernPitch(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)**2/(a*b)

def kernVolume(v1,v2):
    #return kernJacc(v1,v2)
    return min(v1,v2)/max(v1,v2)

def create_random_graph(n):
    G = Graph(loops=False)
    G.add_vertex(1)
    for k in range(2,n+1):
        vert = [v for v in G.vertices()]
        G.add_vertex(k)
        for v in vert:
            prob = 1.0/v*k/sigma(k)
            p = randint(1,100)/100.0
            #print(prob,p)
            if p <= prob and k%v==0:
                G.add_edge(v,k)
    return G

def kernAdd(t1,t2,alphaPitch=0.25):
    pitch1,volume1,isPause1 = t1
    pitch2,volume2,isPause2 = t2
    #return 1.0/3*(1-alphaPitch)*kernPause(isPause1,isPause2)+alphaPitch*kernPitch(pitch1,pitch2)+1.0/3*(1-alphaPitch)*kernDuration(duration1,duration2)+1.0/3*(1-alphaPitch)*kernVolume(volume1,volume2)
    apa = alphaPitch["pause"]
    api = alphaPitch["pitch"]
    avo = alphaPitch["volume"]
    if np.abs(apa+api+avo-1)<10**-5:
        return apa*kernPause(isPause1,isPause2)+api*kernPitch(pitch1,pitch2)+avo*kernVolume(volume1,volume2)
    else:
        return None

def kern(zz0,alphaPitch={"pitch":1,"volume":2,"pause":3}):
    return lambda t1,t2: kernAdd(zz0[int(t1[0])],zz0[int(t2[0])],alphaPitch)


def distKern1(x,y,alphaPitch={"pitch":1,"volume":2,"pause":3}):
    #print(alphaPitch)
    return np.sqrt(2-2*kern(alphaPitch)(x,y))

def distKern(kern):
    return lambda a,b : np.sqrt(2*(1-kern(a,b)))

def writePitches(fn,inds,tempo=82,instrument=[0,0],add21=True,start_at= [0,0],durationsInQuarterNotes=False):
    from MidiFile import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = len(inds)
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(ni):
        MyMIDI.addProgramChange(k,k,0,instrument[k])


    times = start_at
    for k in range(len(inds)):
        channel = k
        track = k
        for i in range(len(inds[k])):
            pitch,duration,volume,isPause = inds[k][i]
            #print(pitch,duration,volume,isPause)
            track = k
            channel = k
            if not durationsInQuarterNotes:
                duration = 4*duration#*maxDurations[k] #findNearestDuration(duration*12*4)            
            #print(k,pitch,times[k],duration,100)
            if not isPause: #rest
                #print(volumes[i])
                # because of median:
                pitch = int(floor(pitch))
                if add21:
                    pitch += 21
                #print(pitch,times[k],duration,volume,isPause)    
                MyMIDI.addNote(track, channel, int(pitch), float(times[k]) , float(duration), int(volume))
                times[k] += duration*1.0  
            else:
                times[k] += duration*1.0
       
    with open(fn, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")  


def run_length_row(row_of_01):
    from itertools import groupby
    ar = row_of_01
    return [(k, sum(1 for i in g)) for k,g in groupby(ar)]

def int_comp_row(row_of_01,by=8):
    ll = divide_row_by(row_of_01,by=by)
    ss = []
    for l in ll:
        rl = run_length_row(l)
        ss.append([v for k,v in rl])
    return ss    

def divide_row_by(row,by):
    ll = []
    n = len(row)
    m = n//by
    #print(m)
    for k in range(m):
        ll.append(row[k*by:((k+1)*by)])
    return ll    

def generateNotes(pitchlist,alphaPitch={"pitch":1,"volume":2,"pause":3},shuffle_notes=True):
    from itertools import product
    from music21 import pitch
    #pitchlist = [p for p in list(range(60-1*octave*12,60+24-1*octave*12))]
    #distmat = np.array(matrix([[np.sqrt(2*(1.0-kernPitch(x,y))) for x in pitchlist] for y in pitchlist]))
    #permutation,distance = tspWithDistanceMatrix(distmat,exact=False)
    #pitchlist = [pitchlist[permutation[k]] for k in range(len(pitchlist))]
    print([pitch.Pitch(midi=int(p)) for p in pitchlist])
    #durationlist = [n for n in durs]
    #if len(durs)>2:
    #    distmat = np.array(matrix([[np.sqrt(2*(1.0-kernDuration(x,y))) for x in durationlist] for y in durationlist]))
    #    permutation,distance = tspWithDistanceMatrix(distmat)
    #    durationlist = [durationlist[permutation[k]] for k in range(len(durationlist))]
    #print(durationlist)
    volumelist = vols = [(128//8)*(k+1) for k in range(8)] #[x*127 for x in [1.0/6.0,1.0/3.0,1.0/2.0,2.0/3.0 ]]
    #distmat = np.array(matrix([[np.sqrt(2*(1.0-kernVolume(x,y))) for x in volumelist] for y in volumelist]))
    #permutation,distance = tspWithDistanceMatrix(distmat)
    #volumelist = [volumelist[permutation[k]] for k in range(len(volumelist))]
    print(volumelist)
    pauselist = [False,True]
    ll = list(product(pauselist,volumelist,pitchlist))
    if shuffle_notes:
        shuffle(ll)
    #distmat = np.array(matrix([[distKern(x,y,alphaPitch) for x in ll] for y in ll]))
    #np.random.seed(43)
    #permutation,distance = tspWithDistanceMatrix(distmat,exact=False)
    #ll = [ll[permutation[k]] for k in range(len(ll))]
    print(len(ll))
    #print(ll)
    pitches = [p[2] for p in ll]
    #durations = [d[0] for d in ll]
    volumes = [v[1] for v in ll]
    isPauses = [p[0] for p in ll]
    print(pitches)
    return pitches,volumes,isPauses    

def getRational(k):
    alpha = 2**(1/12.0)
    x = RDF(alpha**k).n(50)
    return x.nearby_rational(max_error=0.01*x)

def get_knn_model_notes(notes,alphaPitch={"pitch":1.0/6.0,"volume":2.0/6.0,"pause":3.0/6.0}):
    #notes = np.array([[x*1.0 for x in n] for n in notes])
    from sklearn.neighbors import NearestNeighbors
    np.random.seed(0)
    pitches,volumes,isPauses = notes
    zz0 = list(zip(pitches,volumes,isPauses))
    #print(zz0)
    nbrs = NearestNeighbors( algorithm='ball_tree',metric=distKern(kern(zz0,alphaPitch=alphaPitch))).fit([[r] for r in range(len(zz0))])
    return nbrs,notes

def findBestMatches(nbrs,new_row,n_neighbors=3):
    distances,indices = nbrs.kneighbors([np.array(new_row)],n_neighbors=n_neighbors)
    dx = sorted(list(zip(distances[0],indices[0])))
    #print(dx)
    indi = [d[1] for d in dx]
    #print(indi)
    #print(distances)
    #distances,indices = nbrs.query([np.array(new_row)],k=n_neighbors)
    return indi


from itertools import groupby

def get_durations_from_graph(G,sorted_reverse,by=16,shuffled=True):
    zz = sorted(list(zip(G.degree_sequence(),G.vertices())),reverse=sorted_reverse)
    #print(sorted(zz))
    if shuffled:
        shuffle(zz)
    A = G.adjacency_matrix(vertices = [z[1] for z in zz])
    ll = []
    for row in A:
        #print(row)
        ll.extend([xx/by for xx in x] for x in (int_comp_row(row,by=by)))
    ss = []
    for l in ll:
        ss.extend(l)
    return ss    



In [189]:
by = 8
N = by*2
G = graphs.RandomBarabasiAlbert(N,8) #create_random_graph(N)

notes = list(generateNotes(pitchlist=range(30,90),shuffle_notes=False))
#shuffle(notes)
nbrs, notes = get_knn_model_notes(notes)

print(list(range(N,0,-1)))
nVoices = 8
durs = []
iinds = []
for nv in range(nVoices):
    d1 = get_durations_from_graph(G,sorted_reverse=False,by=by,shuffled=True)
    durs.append(d1)
    iinds.append([])


zz0 = list(zip(*notes))


notes1 = []
for d in range(nVoices):
    startNote = [0]
    for k in range(len(durs[d])):
        duration1 = durs[d][k]
        #v = int(sum(Integer(k+1).digits(2)))+1
        v = euler_phi(euler_phi(k+d+1))
        print(v)
        nextNote = findBestMatches(nbrs,startNote,n_neighbors=v)
        notes1.extend((nextNote))
        startNote = [nextNote[0]]
        pitch,volume,isPause = zz0[notes1.pop(0)]
        note1 = pitch,duration1*1.0,volume,isPause
        print(note1)
        iinds[d].append(note1)

    
print(iinds)   

writePitches(fn="./midi/graph_"+str(N)+"-"+str(by)+"_"+str(nVoices)+".mid",inds=iinds,tempo=120,instrument=nVoices*[0],add21=True,start_at= nVoices*[0],durationsInQuarterNotes=False)
    

[<music21.pitch.Pitch F#1>, <music21.pitch.Pitch G1>, <music21.pitch.Pitch G#1>, <music21.pitch.Pitch A1>, <music21.pitch.Pitch B-1>, <music21.pitch.Pitch B1>, <music21.pitch.Pitch C2>, <music21.pitch.Pitch C#2>, <music21.pitch.Pitch D2>, <music21.pitch.Pitch E-2>, <music21.pitch.Pitch E2>, <music21.pitch.Pitch F2>, <music21.pitch.Pitch F#2>, <music21.pitch.Pitch G2>, <music21.pitch.Pitch G#2>, <music21.pitch.Pitch A2>, <music21.pitch.Pitch B-2>, <music21.pitch.Pitch B2>, <music21.pitch.Pitch C3>, <music21.pitch.Pitch C#3>, <music21.pitch.Pitch D3>, <music21.pitch.Pitch E-3>, <music21.pitch.Pitch E3>, <music21.pitch.Pitch F3>, <music21.pitch.Pitch F#3>, <music21.pitch.Pitch G3>, <music21.pitch.Pitch G#3>, <music21.pitch.Pitch A3>, <music21.pitch.Pitch B-3>, <music21.pitch.Pitch B3>, <music21.pitch.Pitch C4>, <music21.pitch.Pitch C#4>, <music21.pitch.Pitch D4>, <music21.pitch.Pitch E-4>, <music21.pitch.Pitch E4>, <music21.pitch.Pitch F4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch 

(49, 0.375000000000000, 16, False)
16
(54, 0.125000000000000, 16, False)
8
(30, 0.375000000000000, 16, False)
24
(42, 0.125000000000000, 16, False)
16
(49, 0.250000000000000, 16, False)
18
(54, 0.125000000000000, 16, False)
16
(58, 0.125000000000000, 16, False)
40
(37, 0.250000000000000, 16, False)
8
(61, 0.125000000000000, 16, False)
32
(66, 0.125000000000000, 16, False)
12
(30, 0.125000000000000, 16, False)
24
(42, 0.500000000000000, 16, False)
16
(49, 0.125000000000000, 16, False)
40
(54, 0.125000000000000, 16, False)
8
(30, 0.125000000000000, 16, False)
24
(42, 0.375000000000000, 16, False)
20
(49, 0.125000000000000, 16, False)
16
(54, 0.125000000000000, 16, False)
22
(58, 0.125000000000000, 16, False)
24
(37, 0.250000000000000, 16, False)
16
(30, 0.125000000000000, 16, False)
32
(42, 0.375000000000000, 16, False)
12
(49, 0.250000000000000, 16, False)
16
(54, 0.125000000000000, 16, False)
16
(30, 0.125000000000000, 16, False)
40
(42, 0.125000000000000, 16, False)
16
(49, 0.25000000

(49, 0.125000000000000, 16, False)
8
(54, 0.125000000000000, 16, False)
8
(58, 0.250000000000000, 16, False)
8
(37, 0.250000000000000, 16, False)
8
(61, 0.250000000000000, 16, False)
8
(66, 0.500000000000000, 16, False)
4
(30, 0.250000000000000, 16, False)
12
(42, 0.500000000000000, 16, False)
6
(49, 0.125000000000000, 16, False)
8
(54, 0.375000000000000, 16, False)
8
(58, 1.00000000000000, 16, False)
16
(37, 0.250000000000000, 16, False)
4
(61, 0.375000000000000, 16, False)
12
(66, 0.375000000000000, 16, False)
8
(68, 0.125000000000000, 16, False)
8
(46, 0.750000000000000, 16, False)
10
(70, 0.125000000000000, 16, False)
22
(35, 0.250000000000000, 16, False)
8
(73, 0.125000000000000, 16, False)
12
(39, 0.125000000000000, 16, False)
8
(77, 0.125000000000000, 16, False)
16
(78, 0.375000000000000, 16, False)
8
(30, 0.375000000000000, 16, False)
24
(42, 0.125000000000000, 16, False)
6
(49, 0.250000000000000, 16, False)
16
(54, 0.125000000000000, 16, False)
8
(58, 0.125000000000000, 16, Fa

(66, 0.125000000000000, 16, False)
32
(30, 0.125000000000000, 16, False)
16
(42, 0.125000000000000, 16, False)
40
(49, 0.750000000000000, 16, False)
16
(54, 0.125000000000000, 16, False)
32
(58, 0.125000000000000, 16, False)
16
(37, 0.125000000000000, 16, False)
40
(61, 0.375000000000000, 16, False)
12
(66, 0.125000000000000, 16, False)
36
(68, 0.125000000000000, 16, False)
32
(46, 0.125000000000000, 16, False)
24
(70, 0.250000000000000, 16, False)
16
(35, 0.250000000000000, 16, False)
48
(73, 0.250000000000000, 16, False)
16
(39, 0.125000000000000, 16, False)
36
(77, 0.125000000000000, 16, False)
20
(78, 0.250000000000000, 16, False)
24
(79, 0.250000000000000, 16, False)
32
(56, 0.125000000000000, 16, False)
64
(80, 0.125000000000000, 16, False)
20
(81, 0.125000000000000, 16, False)
2
(30, 0.500000000000000, 16, False)
1
(42, 0.500000000000000, 16, False)
2
(49, 0.125000000000000, 16, False)
2
(54, 0.125000000000000, 16, False)
2
(58, 0.500000000000000, 16, False)
2
(37, 0.12500000000

(77, 0.125000000000000, 16, False)
16
(78, 0.750000000000000, 16, False)
16
(30, 0.125000000000000, 16, False)
8
(42, 0.125000000000000, 16, False)
20
(49, 0.250000000000000, 16, False)
16
(54, 0.375000000000000, 16, False)
20
(58, 0.250000000000000, 16, False)
8
(37, 0.125000000000000, 16, False)
24
(61, 1.00000000000000, 16, False)
8
(66, 0.125000000000000, 16, False)
24
(68, 0.125000000000000, 16, False)
12
(46, 0.375000000000000, 16, False)
16
(70, 0.250000000000000, 16, False)
12
(35, 0.125000000000000, 16, False)
16
(73, 0.500000000000000, 16, False)
8
(39, 0.125000000000000, 16, False)
24
(77, 0.125000000000000, 16, False)
16
(78, 0.125000000000000, 16, False)
18
(79, 0.125000000000000, 16, False)
16
(56, 0.125000000000000, 16, False)
40
(30, 0.625000000000000, 16, False)
8
(42, 0.125000000000000, 16, False)
32
(49, 0.125000000000000, 16, False)
12
(54, 0.750000000000000, 16, False)
24
(58, 0.125000000000000, 16, False)
16
(37, 0.125000000000000, 16, False)
40
(61, 0.12500000000

(66, 0.125000000000000, 16, False)
4
(68, 0.125000000000000, 16, False)
12
(46, 0.125000000000000, 16, False)
4
(70, 0.125000000000000, 16, False)
8
(35, 0.250000000000000, 16, False)
8
(73, 0.125000000000000, 16, False)
8
(39, 0.125000000000000, 16, False)
8
(77, 0.375000000000000, 16, False)
8
(78, 1.00000000000000, 16, False)
4
(79, 0.125000000000000, 16, False)
12
(56, 0.125000000000000, 16, False)
6
(80, 0.250000000000000, 16, False)
8
(81, 0.125000000000000, 16, False)
8
(34, 0.250000000000000, 16, False)
16
(82, 0.125000000000000, 16, False)
4
(47, 0.625000000000000, 16, False)
12
(85, 0.375000000000000, 16, False)
8
(30, 0.500000000000000, 16, False)
8
(42, 0.125000000000000, 16, False)
10
(49, 0.125000000000000, 16, False)
22
(54, 0.250000000000000, 16, False)
8
(58, 0.250000000000000, 16, False)
12
(37, 0.125000000000000, 16, False)
8
(61, 0.125000000000000, 16, False)
16
(66, 0.125000000000000, 16, False)
8
(68, 0.375000000000000, 16, False)
24
(46, 0.125000000000000, 16, Fa